In [1]:
import os
import logging
import boto3
import pandas as pd
import psycopg2
from dotenv import load_dotenv
from io import StringIO
import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
# Set up environment variables

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load environment variables from .env file
load_dotenv()

# Access environment variables
DB_USERNAME = os.getenv('BDI_DB_USERNAME')
DB_PASSWORD = os.getenv('BDI_DB_PASSWORD')
DB_HOST = os.getenv('BDI_DB_HOST')
DB_PORT = os.getenv('BDI_DB_PORT', '5432')
DB_NAME = os.getenv('BDI_DB_NAME', 'postgres')
AWS_REGION = os.getenv('AWS_REGION')
S3_BUCKET = 'financy-ai'
S3_PREFIX = 'financy'

In [ ]:
# AWS S3 client
s3_client = boto3.client('s3', region_name=AWS_REGION)

In [2]:
def read_json_from_s3(bucket_name, prefix):
    try:
        # List files in the specified bucket with the specified prefix
        response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        files = [file['Key'] for file in response['Contents'] if file['Key'].endswith('.json')]
        
        if not files:
            logging.info("No JSON files found in the bucket.")
            return None
        
        # Processing the fist JSON file found
        json_file_key = files[0]
        json_object = s3_client.get_object(Bucket=bucket_name, Key=json_file_key)
        json_data = json_object['Body'].read().decode('utf-8')
        
        # Load JSON data into Python dictionary
        data = json.loads(json_data)
        
        # Convert to DataFrame if JSON data is structured for tabular format
        if isinstance(data, list):  # Check if data is a list of dictionaries
            df = pd.DataFrame(data)
            logging.info(f"JSON data from {json_file_key} loaded into DataFrame.")
            return df
        else:
            logging.info(f"JSON data from {json_file_key} loaded as dictionary.")
            return data
    except Exception as e:
        logging.error("Error reading JSON from S3: ", exc_info=True)
        return None

In [ ]:
def json_to_postgres(df: pd.DataFrame, ticker: str) -> str:
    try:
        logging.info("Connecting to the PostgreSQL database...")
        conn = psycopg2.connect(**db_credentials)
        cur = conn.cursor()
        logging.info("Successfully connected to the database.")

        # Create table name dynamically based on ticker
        table_name = f"{ticker.lower()}"
        logging.info(f"Preparing to create or confirm existence of table '{table_name}'.")

        # Define the SQL for table creation
        create_table_query = f"""
            CREATE TABLE IF NOT EXISTS {table_name} (
                "id" SERIAL PRIMARY KEY,
                "end" DATE,
                "val" BIGINT,
                "accn" VARCHAR(255),
                "fy" VARCHAR(10),
                "fp" VARCHAR(10),
                "form" VARCHAR(10),
                "filed" DATE,
                "frame" VARCHAR(255),
                "ticker" VARCHAR(10),
                "type" VARCHAR(255),
                "start" DATE
            );
        """
        cur.execute(create_table_query)
        logging.info(f"Table '{table_name}' created or already exists.")

In [ ]:
# Create DB Credentials dictionary

db_credentials = {
    'dbname': DB_NAME,
    'user': DB_USERNAME,
    'password': DB_PASSWORD,
    'host': DB_HOST,
    'port': DB_PORT
}

In [ ]:
def processor:
    # Access RDS database

    # Create table inside the RDS
    
    # Read .json files from the s3 bucket

    # Store the .json files inside the database
    
    return